# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

1. Открываем файл с данными и изучаем его. /datasets/users_behavior.csv
2. Разделяем исходные данные на обучающую, валидационную и тестовую выборки.
3. Исследуем качество разных моделей, меняя гиперпараметры.
4. Проверяем качество модели на тестовой выборке.
5. Дополнительное задание: проверяем модели на вменяемость. 

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


Открываем файл и просматриваем таблицу. В столбце 'is_ultra' будет как признак. Считаем что у нас бинарная классификация.

In [2]:
features = df.drop(['is_ultra'], axis=1 )
target = df['is_ultra']

print(features.shape)
print(target.shape)

(3214, 4)
(3214,)


Отделяем признак от остальных данных.

## Разбейте данные на выборки

In [3]:
features_df, features_valid, target_df, target_valid = train_test_split(features, target, test_size = 0.2, random_state=12345)

print(features_df.shape)
print(features_valid.shape)

(2571, 4)
(643, 4)


In [4]:
features_train, features_test, target_train, target_test = train_test_split(features_df, target_df, test_size = 0.25, random_state=12345)

print(features_train.shape)
print(features_test.shape)

(1928, 4)
(643, 4)


Разбиваем данные на выборки.

## Исследуйте модели

In [5]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state = 12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid  = model.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7480559875583204
max_depth = 2 : 0.7838258164852255
max_depth = 3 : 0.7869362363919129
max_depth = 4 : 0.7869362363919129
max_depth = 5 : 0.7884914463452566


Исследуем модель Дерева Решений.

In [6]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model 
        best_result = result

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129


Модель Случайный Лес.

In [7]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid) 

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.702954898911353


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Логистическая Регрессия.

Логистическая регресиия оказалась саммая непродуктивная. Останавливаем свой выбор на модели Дерева Решений.

## Проверьте модель на тестовой выборке

In [9]:
model = DecisionTreeClassifier(random_state = 12345, max_depth=3)
model.fit(features_train, target_train)
predictions_test  = model.predict(features_test)

print('accuracy =' ,accuracy_score(target_test, predictions_test))

accuracy = 0.7651632970451011


## (бонус) Проверьте модели на адекватность

In [10]:
from sklearn.dummy import DummyClassifier

In [11]:
frequent_clf = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)

In [12]:
freq_pred = frequent_clf.predict(features_test)

In [13]:
uniform_clf = DummyClassifier(strategy='uniform').fit(features_train, target_train)

In [14]:
uniform_pred = uniform_clf.predict(features_test)

In [15]:
frequent_clf.score(features_test, target_test)

0.6889580093312597

In [16]:
uniform_clf.score(features_test, target_test)

0.5038880248833593

Наша мадель оказалась лучше!

## Вывод

Мы создали модель по данным оператора "Мегалайн". Сначала резделили датасет на 3 части:обучающую(25%), валидационную(25%) и тестовую выборку(50%). Дальше мы исслндовали 3 различные модели меняя гипперпараметры: модель Дерева Решений, Случайный Лес и модель Логистическая Регрессия. Из трех моделей, Логистическая регресиия оказалась саммая непродуктивная. Останавливаем свой выбор на модели Дерева Решений. После мы проверяем выбранную модель на тестовой выборке и получаем accuracy = 0.7651632970451011, при задданом минимальном значении в 0,75. 

В пункте №5 проверяем нашу модель на аддекватность с помощью DummyClassifier-это классификатор, который делает прогнозы с использованием простых правил.Этот классификатор полезен в качестве простой основы для сравнения с другими (реальными) классификаторами. Не используется его для решения реальных проблем. Как мы смогли пронаблюдать, наша модель намного лучше!